## Stereo-seq data can be aggregated by bin4 or larger and converted for STHD-compatible anndata through stereopy

In [ ]:
#pip install stereopy
import stereopy as st
import squidpy as sq


In [ ]:

def stereoseq_to_STHD( spatial_path, counts_data, stereo_data_name, full_res_image_path, bin_size = 4, savepath = './'):
    ## Default VisiumHD. Adding stereo-seq compatibility for bin4 or larger. Note that STHD fast_plot would not work due to large number of datapoints.
    print("converting stereoseq gef into STHD compatible. Use load_type = 'crop' in loading.")
    print("bin_size will affect neighbor sizes, recommend bin_size=4 or larger to fit current n_ring setting")
    print("Loading this converted anndata using another mode in sthdio: load_type = 'preload' . ")
    # Use stereopy’s IO to convert Stereo-seq data into an AnnData object.
    # (Based on https://stereopy.readthedocs.io/en/latest/api.html#stereo.io.stereo_to_anndata)
    # Author-conversion: Zijia Tang
    
    if stereo_data_name is None:
        print("Warning! Default name set to Mouse_brain_data.")
        stereo_data_name = 'Mouse_brain_data'
    
    data = st.io.read_gef(file_path= spatial_path + counts_data, bin_size = bin_size)
    adata = st.io.stereo_to_anndata(data, image = full_res_image_path, im_library_id = stereo_data_name)

    adata.obs['array_row'] = adata.obs['y']
    adata.obs['array_col'] = adata.obs['x']

    
    # conversion to 3-channel RGB image
    if adata.uns['spatial'][stereo_data_name]['images']['hires'].shape == 2:
        print("For image compatibility, converting 2-channel image to 3-channel")
        narr = np.zeros((self.adata.uns['spatial'][stereo_data_name]['images']['hires'].shape[0], adata.uns['spatial'][stereo_data_name]['images']['hires'].shape[1], 3), dtype = np.uint8)
        narr[:,:,2] = self.adata.uns['spatial'][stereo_data_name]['images']['hires']
        adata.uns['spatial'][stereo_data_name]['images']['hires'] = narr
        
        narr = np.zeros((self.adata.uns['spatial'][stereo_data_name]['images']['lowres'].shape[0], adata.uns['spatial'][stereo_data_name]['images']['lowres'].shape[1], 3), dtype = np.uint8)
        narr[:,:,2] = self.adata.uns['spatial'][stereo_data_name]['images']['lowres']
        adata.uns['spatial'][stereo_data_name]['images']['lowres'] = narr
    return(adata)